## 1. How to load our images
### 1.1 MNIST-like load_data function
### 1.2 ImageDataGenerator
### 1.3 Dataframes

## 2. MNIST
## 3. Data augmentation
## 4. Pre-trained neural networks
## 5. RGB or grayscale
## 6. Conclusion

# Classifying album covers to genres

In the project we try to classify album covers to genres. The dataset has 5022 pictures, sepreated in five categories. The categories are Rock, Pop, Electronic, Jazz and HipHop. One of the challenges of classifying album covers is that there is no systematic which can be applied on covers. For example when we try to classify a picture as dog or cat, we humans can clearly se the difference. And the machine learning algorithm can try to detected shapes that are common for cats and dogs. But when humans look at an album cover, provided that they don't have any prior knowledge, it's classify an album cover. That first thought about the dataset leads to an early thesis that we probalby need an pre-trained neural network for solving this task appropriate.
Our approach is to start with traditional machine learning methods, like using an convolutional neural network inspired by the architecture used for classifying handwritten didgets from MNIST. 
After that we have an baseline for comparing with other methods and possible improvements.

# 1. How to load images

The dataset consists out of 5022 images seperated in the five categories Rock, Pop, Electronic, Jazz and HipHop.
For each category exists one folder with the associated images. We use three different approaches to load the data.
Our first approach is inspired by the MNIST dataset and we wrote our own `load_data` function. The second approach is the use of an `ImageDataGenerator` that needs a specific directory structure. And the last one is to use dataframes, which are csv files.